In [4]:
# Import Packages

import pandas as pd
import numpy as np
import requests
from requests import get
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

In [5]:
url = "https://www.sciencedaily.com/news/health_medicine/heart_disease/"

In [6]:
# Error shows up as it thinks its a bot
headers2 = {"Accept-Language": "en-US, en;q=0.5"}
result = Request(url, headers=headers2)
webpage = urlopen(result).read()

HTTPError: HTTP Error 403: Forbidden

In [8]:
# use crawl agent
headers1 = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
results = Request(url, headers=headers1)
webpage = urlopen(results).read()
webpage

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta charset="utf-8" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n<meta id="viewport" name="viewport" content="width=device-width, initial-scale=1" />\n<title>Heart Disease News -- ScienceDaily</title>\n<meta name="description" content="Read current medical research on risk factors, causes and prevention of heart disease, strokes, and peripheral arterial disease. What new treatment options are under development?" id="metasummary" />\n<meta name="keywords" content="heart disease; vascular; angioplasty; artery hardening ; cardiac rehabilitation; coronary arteries; narrowing arteries; clogging of the arteries; chronic obstructive pulmonary disease; heart attack risk; stroke; heart risk; obese; bypass; copd; stent; stroke; blood pressure; high blood pressure; hypertension; trigylcerides; cardiopulmonary; cardiovascular; clonidine; hypertension; high nighttime blood pressure; healing the heart; aneurysms; aneurysm formation; athe

In [ ]:
#Parse webpage
soup = BeautifulSoup(webpage,"lxml")

In [ ]:
#Extract headlines
import re
for item in soup.findAll(True, {"class": re.compile("^(tab-content)$")}):
    if 'tab-content' in item.attrs['class'] and 'hero' not in    item.attrs['class']:
        headlines = item
headlines

In [ ]:
#Extract Titles
titles = []
h3_tag = headlines.find_all("h3", class_="latest-head") 
for x in h3_tag:
  for a in x.find_all("a"):
    title = []
    text1 = a.get_text()
    title.append(text1)
    titles.append(title)
titles = pd.DataFrame(titles, columns=['Title'])

In [ ]:
titles.head()

In [ ]:
#Extract dates
dates_descrptions = []
for y in headlines.find_all('div', class_="latest-summary"):
  strip = y.text.strip()
  date_description = []
  date_description.append(strip)
  dates_descrptions.append(date_description)
details = pd.DataFrame(dates_descrptions, columns=['Details']) 

In [ ]:
details.head()

In [ ]:
heart_df = pd.concat([titles, details], axis=1)
heart_df.head()

In [ ]:
#Split ‘Details’ column into two columns — — ‘Date’ and ‘Description’
heart_df[['Date','Description']] = heart_df.Details.apply(lambda x: pd.Series(str(x).split("")))
# drop 'Details' columns
heart_df = heart_df.drop(['Details'], axis=1)
heart_df.head()

In [ ]:
#Rename Columns
# heart_health_ind.head(5)
heart_health_ind_df=pd.DataFrame(heart_df)
heart_health_ind_df.rename(columns={'Title': 'Heart_Disease_Desc'}, inplace=True)
heart_health_ind_df

In [ ]:
#Dropping Duplicates
len_1 = heart_health_ind_df.shape[0]
heart_health_ind1 = heart_health_ind_df.drop_duplicates()
len_2 = heart_health_ind1.shape[0]
print(len_1, len_2)

In [ ]:
#Dataframe with no duplicate for further analysis
import pandas_profiling as pp
heart_health_ind_df1 = pd.DataFrame(heart_health_ind_df)

pp.ProfileReport(heart_health_ind_df1)

In [ ]:
# Counting
heart_health_ind_df1.Heart_Disease_Desc.value_counts()

In [ ]:
#5 - Group By

Heart_limit_low_Group_Count = (heart_health_ind_df1.groupby('Heart_Disease_Desc').agg(count = ('Description', 'size')))

In [ ]:
Heart_limit_low_Group_Count